In [1]:
import pandas as pd

In [2]:
for_mapbox = pd.DataFrame(columns=['name', 'latitude', 'longitude', 'phone_number', 'additional_info', 'type'])

In [80]:
se = pd.read_csv('Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv')
se = se.sort_values(by='COMMUNITY AREA NAME')


import geopandas as gpd
df = gpd.read_file('Boundaries - Neighborhoods.geojson')

df['center'] = df['geometry'].apply(lambda x: str((x.centroid)).strip('0 POINT '))

df['longitude'] = df.center.apply(lambda x: float(x.split(" ")[0].strip("(")))
df['latitude'] = df.center.apply(lambda x: float(x.split(" ")[1].strip(")")))

df = df[['pri_neigh', 'sec_neigh', 'longitude', 'latitude']]
df = df.sort_values(by='pri_neigh').head(80)

se = se.merge(df, left_on='COMMUNITY AREA NAME', right_on='pri_neigh')

In [81]:
se.to_csv('socioeconomic.csv')

In [4]:
housing = pd.read_csv('Affordable_Rental_Housing_Developments.csv')
housing = housing[['Property Name', 'Latitude', 'Longitude', 'Phone Number', 'Property Type']]
housing.columns=['name', 'latitude', 'longitude', 'phone_number', 'additional_info']
housing['type'] = "Affordable Housing"
housing.additional_info = "Property Type: " + housing.additional_info

housing.head()

,name,latitude,longitude,phone_number,additional_info,type
0,Winthrop Apts.,41.995015,-87.658516,773-477-7070,Property Type: Multifamily,Affordable Housing
1,Resurrection Homes,41.855133,-87.703067,312-248-8355,Property Type: Multifamily,Affordable Housing
2,Renaissance St. Luke Greenview Place,41.940531,-87.666383,773-525-1501,Property Type: Senior,Affordable Housing
3,All Saints Senior Residence,41.681590,-87.622628,312-655-7440,Property Type: Senior,Affordable Housing
4,Victory Center of Roseland,41.703891,-87.620771,773-468-6400,Property Type: Senior,Affordable Housing


In [5]:
grocery = pd.read_csv('IL.csv')
grocery = grocery[grocery.CITY.isin(['Chicago', 'Chiacgo', 'CHICAGO'])]
grocery = grocery[['STORE_NAME', 'ADDRESS', 'longitude', 'latitude']]
grocery.to_csv('grocery.csv')
# add phone numbers from YELP

In [6]:
libraries = pd.read_csv('Libraries_-_Locations__Hours_and_Contact_Information.csv')
libraries['latitude'] = libraries.LOCATION.apply(lambda x: float(x.strip("()").split(",")[0]))
libraries['longitude'] = libraries.LOCATION.apply(lambda x: float(x.strip("()").split(",")[1]))
libraries = libraries[['NAME ', 'latitude', 'longitude', 'PHONE', 'HOURS OF OPERATION']]
libraries.columns=['name', 'latitude', 'longitude', 'phone_number', 'additional_info']
libraries.additional_info = "Hours: " + libraries.additional_info
libraries['type'] = "Library"

libraries.head()

,name,latitude,longitude,phone_number,additional_info,type
0,South Shore,41.761555,-87.562084,Closed for Construction,NaN,Library
1,Albany Park,41.975456,-87.714090,(773) 539-5450,"Hours: Sun., Closed; Mon. & Wed., 10-6; Tue. &...",Library
2,Altgeld,41.654730,-87.602230,(312) 747-3270,"Hours: Sun., Closed; Mon. & Wed., Noon-8; Tue....",Library
3,Archer Heights,41.801214,-87.726491,(312) 747-9241,"Hours: Sun., Closed; Mon. & Wed., Noon-8; Tue....",Library
4,Austin,41.889272,-87.765712,(312) 746-5038,"Hours: Sun., Closed; Mon. & Wed., Noon-8; Tue....",Library


In [7]:
spqr = pd.ExcelFile('SQRP_18_19.xls')
elem = pd.read_excel(spqr, "elem", header=0, skiprows=1)
high = pd.read_excel(spqr, "high")
combo = pd.read_excel(spqr, "combo")
option = pd.read_excel(spqr, "option")

all_schools = elem
for s in ['high', 'combo', 'option']:
    all_schools = all_schools.append(pd.read_excel(spqr, s, header=0))
    print(all_schools.shape)

(592, 104)
(621, 179)
(663, 182)


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [9]:
all_schools2 = all_schools[['School ID', 'SY 2018-2019 SQRP Rating']]
all_schools2.columns = ['School_ID', 'additional_info']
all_schools2 = all_schools2.dropna(subset=['School_ID'])
all_schools2.School_ID = all_schools2.School_ID.apply(lambda x: int(x))

In [10]:
s = pd.read_csv('Chicago_Public_Schools_-_School_Profile_Information_SY1617 (1).csv')
schools = s.merge(all_schools2, on='School_ID', how='left')
schools = schools[['Long_Name', 'School_Latitude', 'School_Longitude', 'Phone', 'additional_info']]
schools.columns=['name', 'latitude', 'longitude', 'phone_number', 'additional_info']
schools['type'] = "School"
schools['additional_info'] = "SQRP Rating: " + schools.additional_info

schools.head()

,name,latitude,longitude,phone_number,additional_info,type
0,Harriet E Sayre Elementary Language Academy,41.914151,-87.798721,7735343351,SQRP Rating: Level 2+,School
1,Ronald E McNair Elementary School,41.897819,-87.746731,7735348980,SQRP Rating: Level 1+,School
2,Charles N Holden Elementary School,41.838027,-87.653792,7735357200,SQRP Rating: Level 1+,School
3,Acero Charter Schools - SPC Daniel Zizumbo,41.810135,-87.730495,7735793470,SQRP Rating: Level 1,School
4,John B Murphy Elementary School,41.950078,-87.716832,7735345223,SQRP Rating: Level 2+,School


In [11]:
for_mapbox = for_mapbox.append(housing)
for_mapbox = for_mapbox.append(libraries)
for_mapbox = for_mapbox.append(schools)
# add grocery

for_mapbox = for_mapbox.fillna('')

In [16]:
for_mapbox.head()

,name,latitude,longitude,phone_number,additional_info,type
0,Winthrop Apts.,41.995015,-87.658516,773-477-7070,Property Type: Multifamily,Affordable Housing
1,Resurrection Homes,41.855133,-87.703067,312-248-8355,Property Type: Multifamily,Affordable Housing
2,Renaissance St. Luke Greenview Place,41.940531,-87.666383,773-525-1501,Property Type: Senior,Affordable Housing
3,All Saints Senior Residence,41.681590,-87.622628,312-655-7440,Property Type: Senior,Affordable Housing
4,Victory Center of Roseland,41.703891,-87.620771,773-468-6400,Property Type: Senior,Affordable Housing
